In [33]:
import pandas as pd
import numpy as np
from prophet import Prophet
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, r2_score

In [34]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [35]:
file_path = '/content/drive/MyDrive/Colab Notebooks/Electric_Consumption_And_Cost__2010_-_Feb_2023__20231212.csv'

In [ ]:
df = pd.read_csv(file_path)

<ipython-input-21-47c250c455f0>:1: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 447849 entries, 0 to 447848
Data columns (total 27 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Development Name    446368 non-null  object 
 1   Borough             447849 non-null  object 
 2   Account Name        447849 non-null  object 
 3   Location            436128 non-null  object 
 4   Meter AMR           447622 non-null  object 
 5   Meter Scope         23867 non-null   object 
 6   TDS #               445111 non-null  float64
 7   EDP                 447849 non-null  int64  
 8   RC Code             447849 non-null  object 
 9   Funding Source      447849 non-null  object 
 10  AMP #               445962 non-null  object 
 11  Vendor Name         447849 non-null  object 
 12  UMIS BILL ID        447849 non-null  int64  
 13  Revenue Month       447849 non-null  object 
 14  Service Start Date  447841 non-null  object 
 15  Service End Date    447841 non-nul

In [ ]:
df.dtypes

Development Name       object
Borough                object
Account Name           object
Location               object
Meter AMR              object
Meter Scope            object
TDS #                 float64
EDP                     int64
RC Code                object
Funding Source         object
AMP #                  object
Vendor Name            object
UMIS BILL ID            int64
Revenue Month          object
Service Start Date     object
Service End Date       object
# days                float64
Meter Number           object
Estimated              object
Current Charges       float64
Rate Class             object
Bill Analyzed          object
Consumption (KWH)     float64
KWH Charges           float64
Consumption (KW)      float64
KW Charges            float64
Other charges         float64
dtype: object

In [ ]:
df.head()

,Development Name,Borough,Account Name,Location,Meter AMR,Meter Scope,TDS #,EDP,RC Code,Funding Source,...,Meter Number,Estimated,Current Charges,Rate Class,Bill Analyzed,Consumption (KWH),KWH Charges,Consumption (KW),KW Charges,Other charges
0,ADAMS,BRONX,ADAMS,BLD 05,NONE,BLD 01 to 07,118.0,248,B011800,FEDERAL,...,7223256,N,15396.82,GOV/NYC/068,Yes,128800.0,7387.97,216.0,2808.0,5200.85
1,ADAMS,BRONX,ADAMS,BLD 05,NONE,BLD 01 to 07,118.0,248,B011800,FEDERAL,...,7223256,N,14556.34,GOV/NYC/068,Yes,115200.0,6607.87,224.0,2912.0,5036.47
2,ADAMS,BRONX,ADAMS,BLD 05,NONE,BLD 01 to 07,118.0,248,B011800,FEDERAL,...,7223256,N,13904.98,GOV/NYC/068,Yes,103200.0,5919.55,216.0,2808.0,5177.43
3,ADAMS,BRONX,ADAMS,BLD 05,NONE,BLD 01 to 07,118.0,248,B011800,FEDERAL,...,7223256,N,14764.04,GOV/NYC/068,Yes,105600.0,6057.22,208.0,2704.0,6002.82
4,ADAMS,BRONX,ADAMS,BLD 05,NONE,BLD 01 to 07,118.0,248,B011800,FEDERAL,...,7223256,N,13729.54,GOV/NYC/068,Yes,97600.0,5598.34,216.0,2808.0,5323.20


In [ ]:
df = df[['Revenue Month', 'Consumption (KWH)']].copy().dropna()
df.head()

,Revenue Month,Consumption (KWH)
0,2010-01,128800.0
1,2010-02,115200.0
2,2010-03,103200.0
3,2010-04,105600.0
4,2010-05,97600.0


In [ ]:
df['Revenue Month'] = pd.to_datetime(df['Revenue Month'], dayfirst=True)

In [ ]:
monthly_consumption = df['Revenue Month'].unique()
mean_consumption = []
for i in monthly_consumption:
    temp = df[df['Revenue Month'] == i]
    mean_val = temp['Consumption (KWH)'].mean()
    temp1 = [i,mean_val]
    mean_consumption.append(temp1)

In [ ]:
monthly_consumption = pd.DataFrame(mean_consumption, columns = ['ds', 'y'])
monthly_consumption

,ds,y
0,2010-01-01,45446.030782
1,2010-02-01,38808.144583
2,2010-03-01,36667.210941
3,2010-04-01,36808.514463
4,2010-05-01,36933.632880
...,...,...
128,2021-07-01,36927.006146
129,2021-08-01,37567.405912
130,2021-09-01,33186.180491
131,2021-10-01,25960.355783


In [ ]:
daily_consumption = pd.DataFrame()
for index, row in monthly_consumption.iterrows():
    num_days = pd.Period(row['ds'], freq = 'M').days_in_month
    for i in range(num_days):
        daily_consumption = daily_consumption.append(
            {
                'ds':row['ds'] + pd.DateOffset(days=i),
                'y': row['y']/30,
            },
            ignore_index=True,
        )
daily_consumption.tail(40)

Streaming output truncated to the last 5000 lines.
<ipython-input-29-e2a52321442d>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  daily_consumption = daily_consumption.append(
<ipython-input-29-e2a52321442d>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  daily_consumption = daily_consumption.append(
<ipython-input-29-e2a52321442d>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  daily_consumption = daily_consumption.append(
<ipython-input-29-e2a52321442d>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  daily_consumption = daily_consumption.append(
<ipython-input-29-e2a52321442d>:5: FutureWarning: The frame.append method is deprecat

,ds,y
4009,2021-10-23,865.345193
4010,2021-10-24,865.345193
4011,2021-10-25,865.345193
4012,2021-10-26,865.345193
4013,2021-10-27,865.345193
4014,2021-10-28,865.345193
4015,2021-10-29,865.345193
4016,2021-10-30,865.345193
4017,2021-10-31,865.345193
4018,2021-12-01,870.351497


In [ ]:
def train_model(growth, seasonality, df):
    model = None

    if seasonality == 'daily':
        model = Prophet (interval_width=0.95, growth=growth, changepoint_prior_scale=30, n_changepoints=3,
        daily_seasonality=False, weekly_seasonality=False, yearly_seasonality=False)
        model.add_seasonality (name='daily', period=1, fourier_order=15)
        model.fit (df)

    elif seasonality == 'monthly':
        model = Prophet (interval_width=0.95, growth=growth, changepoint_prior_scale=30, n_changepoints=3,
        daily_seasonality=False, weekly_seasonality=False, yearly_seasonality=False)
        model.add_seasonality (name='monthly', period=30.5, fourier_order=12)
        model.fit(df)

    elif seasonality == 'annual':
        model = Prophet (interval_width=0.95, growth=growth, changepoint_prior_scale=30, n_changepoints=2,
        daily_seasonality=False, weekly_seasonality=False, yearly_seasonality=False)
        model.add_seasonality (name='annual', period=365.25, fourier_order=20)
        model.fit (df)
    return model

In [ ]:
def get_forecast(model, time_period):
    future_df = model.make_future_dataframe(periods=time_period)
    forecast = model.predict(future_df)
    return future_df, forecast
def evaluate_forecast(forecast, actual, time_period):
    y_true = actual
    y_pred = forecast[:(len(forecast)-time_period)]

    mae = mean_absolute_error(y_true, y_pred)
    mape = mean_absolute_percentage_erroe(y_true, y_pred)
    reg_score = r2_score(y_true, y_pred)

    print("mean absolute error: ", mae)
    print("mean absolute percenatge error: ", mape)
    print("regression score: ", reg_score)
    plt.plot(y_pred, c='black', label='Predicted')
    plt.plot(y_true, c='blue', label='Actual')
    plt.legend(loc='lower right')
    plt.show()

def plot_graph(data, label):
    plt.plot(data, c='red', label=label)
    plt.legend(loc='lower right')
    plt.show()

In [ ]:
model_monthly = train_model(growth='linear', seasonality='monthly', df=monthly_consumption)
future_monthly, forecast_monthly = get_forecast(model=model_monthly, time_period=1)
forecast_monthly.head()

DEBUG:cmdstanpy:input tempfile: /tmp/tmp6kku2x8m/nxzu4_dm.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp6kku2x8m/y2kxb93y.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=2746', 'data', 'file=/tmp/tmp6kku2x8m/nxzu4_dm.json', 'init=/tmp/tmp6kku2x8m/y2kxb93y.json', 'output', 'file=/tmp/tmp6kku2x8m/prophet_model34pgtsna/prophet_model-20231212135528.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
13:55:28 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
13:55:28 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,monthly,monthly_lower,monthly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2010-01-01,43240.993151,33631.365917,58554.090324,43240.993151,43240.993151,2096.122332,2096.122332,2096.122332,2096.122332,2096.122332,2096.122332,0.0,0.0,0.0,45337.115483
1,2010-02-01,42868.342768,25729.243685,51104.811797,42868.342768,42868.342768,-4012.512320,-4012.512320,-4012.512320,-4012.512320,-4012.512320,-4012.512320,0.0,0.0,0.0,38855.830448
2,2010-03-01,42531.755325,34492.944376,58446.537351,42531.755325,42531.755325,3696.144222,3696.144222,3696.144222,3696.144222,3696.144222,3696.144222,0.0,0.0,0.0,46227.899547
3,2010-04-01,42159.104941,33174.086263,57568.573740,42159.104941,42159.104941,3290.812647,3290.812647,3290.812647,3290.812647,3290.812647,3290.812647,0.0,0.0,0.0,45449.917588
4,2010-05-01,41798.475537,33477.460358,56917.500957,41798.475537,41798.475537,3696.144222,3696.144222,3696.144222,3696.144222,3696.144222,3696.144222,0.0,0.0,0.0,45494.619759
